TODO: continue this

In [ ]:
import torch as tn
import torch.nn as nn

try:
    import torchtt as tntt
except:
    print('Installing torchTT...')
    %pip install git+https://github.com/ion-g-ion/torchTT
    import torchtt as tntt

$\mathbb{R}^{16} \times\mathbb{R}^{16} \times\mathbb{R}^{16} \times\mathbb{R}^{16} \underset{\mathcal{LTT}^1}{\longrightarrow} \mathbb{R}^8 \times\mathbb{R}^8 \times\mathbb{R}^8 \times\mathbb{R}^8 \underset{\mathcal{LTT}^2}{\longrightarrow} \mathbb{R}^4 \times\mathbb{R}^4 \times\mathbb{R}^4 \times\mathbb{R}^4  \underset{\mathcal{LTT}^3}{\longrightarrow}  \mathbb{R}^2 \times\mathbb{R}^4 \times\mathbb{R}^2 \times\mathbb{R}^4 \underset{\mathcal{L}^4}{\longrightarrow} \mathbb{R}^{10}$

In [ ]:
class BasicTT(nn.Module):
    def __init__(self):
        super().__init__()
        self.ttl1 = tntt.nn.LinearLayerTT([16,16,16,16], [8,8,8,8], [1,3,3,3,1])
        self.ttl2 = tntt.nn.LinearLayerTT([8,8,8,8], [4,4,4,4], [1,2,2,2,1])
        self.ttl3 = tntt.nn.LinearLayerTT([4,4,4,4], [2,4,2,4], [1,2,2,2,1])
        self.linear = nn.Linear(64, 10, dtype = tn.float32)

    def forward(self, x):
        x = self.ttl1(x)
        x = tn.relu(x)
        x = self.ttl2(x)
        x = tn.relu(x)
        x = self.ttl3(x)
        x = tn.relu(x)
        x = tn.reshape(x,[-1,64])
        return self.linear(x)

In [ ]:
model = BasicTT()
print('Number of trainable parameters:', len(list(model.parameters())))

In [ ]:
input = tn.rand((16,16,16,16), dtype = tn.float32)
pred = model.forward(input)

input_batch = tn.rand((1000,16,16,16,16), dtype = tn.float32)
label_batch = tn.rand((1000,10), dtype = tn.float32)
%time pred = model.forward(input_batch)

In [ ]:
# if tn.cuda.is_available():
#     model_gpu = model.cuda()
#     input_batch_gpu = input_batch.cuda()
# 
#     %time pred = model_gpu.forward(input_batch_gpu+2)
#     print(pred.device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = tn.optim.SGD(model.parameters(), lr=0.0000001, momentum=0.9)

In [ ]:
for epoch in range(10):  

    optimizer.zero_grad()

    outputs = model(input_batch)
    loss = criterion(outputs, label_batch)
    loss.backward()
    optimizer.step()

    # print statistics
    print('Epoch %d, loss %e'%(epoch+1,loss.item()))


print('Finished Training')

In [ ]:
model
